In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

from SpectralLayer import*
from utilsSimpleConv2D import*
from spectralconvolutions import *

from tensorflow.keras.layers import Layer
from typing import Tuple,List,Any,Dict

In [2]:
## Data

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test=x_train.reshape(-1,28,28,1), x_test.reshape(-1,28,28,1)

x_train, x_test = x_train / 255.0, x_test / 255.0

flat_train = np.reshape(x_train, [x_train.shape[0], 28 * 28])
flat_test = np.reshape(x_test, [x_test.shape[0], 28 * 28])

In [4]:
## Class PaddingJacobiens

In [5]:
## class SpecConv2D

In [6]:
## Create layer specConv2D

In [7]:
kernel_size=5
inputs=tf.keras.layers.Input(shape=(28,28,2))
layer1=PaddingJacobiens(kernel_size=kernel_size,
                       strides=1,
                        padding='VALID')
outputs1=layer1(inputs)
#print(f"Outputs1: {outputs1}")
layer2=SpecConv2D(filters=1,
                 kernel_size=kernel_size,
                 use_lambda_in=True,
                 use_bias=False,
                 activation="relu")
outputs2=layer2(outputs1)
print(f"Outputs2: {outputs2}")

Outputs2: KerasTensor(type_spec=TensorSpec(shape=(None, 24, 24, 2), dtype=tf.float32, name=None), name='spec_conv2d/Relu:0', description="created by layer 'spec_conv2d'")


In [8]:
## Build model

In [ ]:
epochs = 10
batch_size=200
accuracy=list()

spectral_config={
    'is_base_trainable' : False,
    'is_diag_start_trainable' : False,
    'is_diag_end_trainable' : True,
}
N2=[20,80,100,500,800,1000,1500,2000]

for i in range(len(N2)):
    print(f"Iteration {i}: nombre de neurones: {N2[i]}")
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(28,28,1)))
    
    model.add(PaddingJacobiens(kernel_size=3, strides=1, padding="VALID"))
    model.add(SpecConv2D(filters=1, kernel_size=3,use_lambda_in=True,use_bias=False, activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2,2)))     
    
    model.add(tf.keras.layers.Flatten())     
    
    model.add(Spectral(N2[i], **spectral_config,use_bias=False, activation=None))
    model.add(Spectral(10, **spectral_config,use_bias=False, activation='softmax'))
           
    opt = tf.keras.optimizers.Adam(learning_rate=0.005)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=0,validation_data=(x_test, y_test))
    accuracy.append(model.evaluate(x_test,y_test,batch_size=32,verbose="auto"))

Iteration 0: nombre de neurones: 20


In [ ]:
score=list()
for i in range(len(N2)):
    score.append(accuracy[i][1])

In [ ]:
## Dense

In [ ]:

accuracy2=list()
N2=[20,80,100,500,800,1000,1500,2000]
epochs = 20
batch_size=200

for i in range(len(N2)):
    print(f"Iteration {i}: nombre de neurones: {N2[i]}")
    model2 = tf.keras.Sequential()
    model2.add(tf.keras.layers.Input(shape=(28,28,1)))
    
    
    model2.add(keras.layers.Conv2D(1,(3,3), strides=1,use_bias=False,padding='VALID',activation="relu"))
    model2.add(keras.layers.MaxPooling2D((2,2)))
    
    model2.add(keras.layers.Flatten())  
    
    
    model2.add(Dense(N2[i],use_bias=False,activation='relu'))
    model2.add(Dense(10, use_bias=False, activation='softmax'))
    
               
    opt = tf.keras.optimizers.Adam(learning_rate=0.005)
    model2.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=0,validation_data=(x_test, y_test))
    accuracy2.append(model2.evaluate(x_test,y_test,batch_size=32,verbose="auto"))

In [ ]:
score2=list()
for i in range(len(N2)):
    score2.append(accuracy2[i][1])